In [1]:
!pip list | grep tensorflow

tensorflow                    2.13.0
tensorflow-datasets           4.9.2
tensorflow-estimator          2.13.0
tensorflow-hub                0.14.0
tensorflow-io-gcs-filesystem  0.33.0
tensorflow-metadata           1.14.0
tensorflow-model-optimization 0.7.5
tensorflow-text               2.13.0


In [2]:
%load_ext autoreload
%autoreload 2
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from src.constants import TARGET_MAX_LENGHT, MAX_LENGHT_SOURCE, FEATURE_COLUMNS
from src.data_utils.dataset import build_datset_train_val, VOCAB_SIZE, LHAND_IDX, LHAND_IDX, start_token_idx, end_token_idx, pre_process, pad_token_idx
from src.prod_models.builder import build_prod_transformer_model_v1
from src.callbacks import get_predefine_callbacks
import optuna
import tensorflow as tf
import numpy as np

TRIALS=5
EPOCHS=10
EPOCHS_PER_TRIAL=10
BATCH_SIZE = 128
TRAIN_SPLIT = 0.7

In [3]:
TARGET_MAX_LENGHT

64

In [4]:
train_dataset, val_dataset = build_datset_train_val(split=TRAIN_SPLIT, batch_size=BATCH_SIZE)

train split: 24576 | val split: 10240


In [5]:
next(iter(val_dataset))[0][1][0]

<tf.Tensor: shape=(64,), dtype=int32, numpy=
array([60, 20, 24, 23, 22, 22,  0, 50, 34, 39, 46, 36, 51, 51, 43, 36,  0,
       49, 46, 32, 35, 61, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59,
       59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59,
       59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59], dtype=int32)>

In [6]:
 model = build_prod_transformer_model_v1(trial=None)

In [7]:
model.decoder.get_casual_attention_mask(tf.constant([[1, 2, 3, 59, 59, 59], [1,2,3,59,59,59]]))

<tf.Tensor: shape=(2, 6, 6), dtype=int32, numpy=
array([[[1, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1]],

       [[1, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1]]], dtype=int32)>

In [8]:
def objective(trial):
    tf.keras.backend.clear_session()
    model = build_prod_transformer_model_v1(trial=trial)
    model.build([(None, MAX_LENGHT_SOURCE, int(FEATURE_COLUMNS.shape[0]/2)), (None, TARGET_MAX_LENGHT)])
    model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS_PER_TRIAL, callbacks=get_predefine_callbacks(model_name="prod_v1"))
    levenshtein = model.evaluate(val_dataset)[-1]
    # val_loss = model.evaluate(val_dataset)[0]
    # return  val_loss
    return  levenshtein

In [9]:
# study = optuna.create_study(direction='minimize', storage="sqlite:///db.sqlite3", study_name="prod_v1")
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=TRIALS, gc_after_trial=True, show_progress_bar=True)

[I 2023-08-24 00:15:20,297] A new study created in memory with name: no-name-c41ae697-b8c2-437b-a7d1-bf848e68a07c


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/10
280/280 [==============================] - 35s 95ms/step - loss: 0.9781 - accuracy: 0.7408 - sparse_levenshtein_v1: 0.2512 - val_loss: 0.8182 - val_accuracy: 0.7630 - val_sparse_levenshtein_v1: 0.2319
Epoch 2/10
280/280 [==============================] - 25s 89ms/step - loss: 0.7962 - accuracy: 0.7666 - sparse_levenshtein_v1: 0.2291 - val_loss: 0.7727 - val_accuracy: 0.7713 - val_sparse_levenshtein_v1: 0.2262
Epoch 3/10
280/280 [==============================] - 25s 90ms/step - loss: 0.7586 - accuracy: 0.7749 - sparse_levenshtein_v1: 0.2225 - val_loss: 0.7444 - val_accuracy: 0.7771 - val_sparse_levenshtein_v1: 0.2205
Epoch 4/10
280/280 [==============================] - 25s 90ms/step - loss: 0.7383 - accuracy: 0.7799 - sparse_levenshtein_v1: 0.2177 - val_loss: 0.7308 - val_accuracy: 0.7818 - val_sparse_levenshtein_v1: 0.2164
Epoch 5/10
280/280 [==============================] - 25s 91ms/step - loss: 0.7275 - accuracy: 0.7826 - sparse_levenshtein_v1: 0.2152 - val_loss: 0.7249

In [10]:
tf.keras.backend.clear_session()
trials = study.best_trials

for index, trial in enumerate(trials):
    print(f"Best model: {index+1}")
    model_name = "prod_v1"

    model = build_prod_transformer_model_v1(trial=trial)

    model.build([(None, MAX_LENGHT_SOURCE, int(FEATURE_COLUMNS.shape[0]/2)), (None, TARGET_MAX_LENGHT)])

    model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS, callbacks=get_predefine_callbacks(model_name=model_name))
    print(model.summary())

    print('validation levenshtein distance: {}'.format(trial.value))
    print("Best hyperparameters: {}".format(trial.params))

    model.load_weights(f"../best_model/prototype/{model_name}")

    print(f"Metrics in Validation: {model.evaluate(val_dataset)}")

Best model: 1
Epoch 1/10
280/280 [==============================] - 30s 83ms/step - loss: 0.9993 - accuracy: 0.7349 - sparse_levenshtein_v1: 0.2570 - val_loss: 0.8217 - val_accuracy: 0.7624 - val_sparse_levenshtein_v1: 0.2316
Epoch 2/10
280/280 [==============================] - 23s 80ms/step - loss: 0.7986 - accuracy: 0.7674 - sparse_levenshtein_v1: 0.2270 - val_loss: 0.7712 - val_accuracy: 0.7737 - val_sparse_levenshtein_v1: 0.2239
Epoch 3/10
280/280 [==============================] - 22s 80ms/step - loss: 0.7583 - accuracy: 0.7756 - sparse_levenshtein_v1: 0.2219 - val_loss: 0.7440 - val_accuracy: 0.7778 - val_sparse_levenshtein_v1: 0.2207
Epoch 4/10
280/280 [==============================] - 23s 80ms/step - loss: 0.7382 - accuracy: 0.7799 - sparse_levenshtein_v1: 0.2183 - val_loss: 0.7354 - val_accuracy: 0.7812 - val_sparse_levenshtein_v1: 0.2176
Epoch 5/10
280/280 [==============================] - 23s 80ms/step - loss: 0.7277 - accuracy: 0.7828 - sparse_levenshtein_v1: 0.2158 - va

In [11]:
from src.data_utils.dataset import char_to_num, num_to_char

In [12]:
target_sequence = [char_to_num[w] for w in ["<"]]

for batch_index, batch in enumerate(val_dataset):
    batch = batch[0]

    sources = batch[0] #batch["source"]
    targets = batch[1] #batch["target"]
    
    print(sources.shape)
    print(targets.shape)

    for index_sample, (source, target) in enumerate(zip(sources, targets)):
        source = tf.expand_dims(source, axis=0)
        target_sequence = [char_to_num[w] for w in ["<"]]
        y_true = "".join([num_to_char[w] for w in target.numpy()])
    
        for i in range(TARGET_MAX_LENGHT):
            next_token = tf.expand_dims(tf.pad(tf.constant(target_sequence),
             [[0, TARGET_MAX_LENGHT-len(target_sequence)]],
              mode='CONSTANT',
               constant_values=pad_token_idx,
                name=None),
                 axis=0)

            print("next target sequence to predict: ", next_token)

            # y_pred = model({"source": source, "target": next_token})
            y_pred = model((source, next_token))

            y_pred = tf.cast(tf.argmax(y_pred, axis=2), dtype=tf.int32)

            print("argmax:", y_pred)

            mask = tf.not_equal(y_pred, pad_token_idx)
            next_token = y_pred[mask][-1].numpy()

            target_sequence.append(next_token)

            print("sequence so far: ", "".join([num_to_char[w] for w in target_sequence]))
            print("Label: ", y_true)

            if num_to_char[next_token]==">":
                break

        print(f"================================={index_sample}=========================================")
        if index_sample==1:
            break

    if batch_index==1:
        break

(128, 128, 42)
(128, 64)
next target sequence to predict:  tf.Tensor(
[[60 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59
  59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59
  59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59]], shape=(1, 64), dtype=int32)
argmax: tf.Tensor(
[[10 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59
  59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59
  59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59]], shape=(1, 64), dtype=int32)
sequence so far:  <+
Label:  <59877 schoettle road>PPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPPP
next target sequence to predict:  tf.Tensor(
[[60 10 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59
  59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59
  59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59]], shape=(1, 64), dtype=int32)
argmax: tf.Tensor(
[[10 20 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 59 5

In [13]:
model.summary()

Model: "finger_spelling_v1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 landmark_embedding_v1 (Lan  multiple                  2107392   
 dmarkEmbeddingV1)                                               
                                                                 
 basic_positional_embedding  multiple                  32256     
 s (BasicPositionalEmbeddin                                      
 gs)                                                             
                                                                 
 transformer_encoder (Trans  multiple                  794631    
 formerEncoder)                                                  
                                                                 
 transformer_decoder (Trans  multiple                  1587981   
 formerDecoder)                                                  
                                                

In [14]:
# # save model

# model.save("../models/prod_v1.keras", save_format="keras")
model.save("../models/prod_v1", save_format="tf")

INFO:tensorflow:Assets written to: ../models/prod_v1/assets


INFO:tensorflow:Assets written to: ../models/prod_v1/assets


In [15]:
# from src.custom.metrics import SparseLevenshtein

# model_loaded = tf.keras.models.load_model("../models/prod_v1.keras", custom_objects={"SparseLevenshtein": SparseLevenshtein})

In [16]:
# #after register class as serializable
# model_loaded = tf.keras.models.load_model("../models/prod_v1.keras")

# TF lite

In [18]:
model.generate(tf.random.uniform(shape=(1, MAX_LENGHT_SOURCE, 42)))

<tf.Tensor: shape=(1, 64), dtype=int32, numpy=
array([[60, 10, 20, 20, 12, 20, 20, 12, 20, 20, 12, 20, 20, 12, 20, 20,
        61, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59,
        59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59,
        59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59]],
      dtype=int32)>

In [19]:
class TFLiteModel(tf.Module):
    def __init__(self, model):
        super(TFLiteModel, self).__init__()
        self.target_start_token_idx = start_token_idx
        self.target_end_token_idx = end_token_idx
        # Load the feature generation and main models
        self.model = model

    @tf.function(input_signature=[tf.TensorSpec(shape=[None, FEATURE_COLUMNS.shape[0]], dtype=tf.float32, name='inputs')])
    def __call__(self, inputs, training=False):
        # Preprocess Data
        x = tf.cast(inputs, tf.float32)

        x = x[None]

        x = tf.cond(tf.shape(x)[1] == 0, lambda: tf.zeros((1, 1, FEATURE_COLUMNS.shape[0])), lambda: tf.identity(x))

        x = x[0]

        x = pre_process(x)
        #shape after [MAX_LENGHT_SOURCE, FEATURE_SIZE]

        x = x[None]

        x = self.model.generate(x)

        x = x[0]
        idx = tf.argmax(tf.cast(tf.equal(x, self.target_end_token_idx), tf.int32))
        idx = tf.where(tf.math.less(idx, 1), tf.constant(2, dtype=tf.int64), idx)
        x = x[1:idx]

        x = tf.one_hot(x, 59)
        return {"outputs": x}

tflitemodel_base = TFLiteModel(model)

In [20]:
keras_model_converter = tf.lite.TFLiteConverter.from_keras_model(tflitemodel_base)
keras_model_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
keras_model_converter.allow_custom_ops = True
keras_model_converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = keras_model_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpi8fa4n73/assets


INFO:tensorflow:Assets written to: /tmp/tmpi8fa4n73/assets


In [22]:
import json

# with open('/kaggle/working/model.tflite', 'wb') as f:
with open("../models/model.tflite", "wb") as f:    
    f.write(tflite_model)

infargs = {"selected_columns" : list(FEATURE_COLUMNS)}

# with open("inference_args.json", "w") as json_file:
with open("../models/inference_args.json", "w") as json_file:
    json.dump(infargs, json_file)

In [23]:
!zip submission.zip  '../models/model.tflite' '../models/inference_args.json'

updating: ../models/model.tflite (deflated 37%)
updating: ../models/inference_args.json (deflated 84%)


# Explore tflite by hand step by step

In [24]:
# when None None in signature
# array([ 1,  1, 42]

interpreter = tf.lite.Interpreter("../models/model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)
print(output_details)

x = tf.random.uniform(shape=(250, FEATURE_COLUMNS.shape[0]), dtype=tf.float32)
print(x.shape)

interpreter.set_tensor(input_details[0]["index"], x)
# not needed now
# interpreter.resize_tensor_input(input_details[0]['index'], [250, 42])

print('tensor details:', interpreter.get_tensor_details())

interpreter.invoke()

In [25]:
# from batch 1

source_batch, target_batch = next(iter(train_dataset))[0]

In [26]:
source_batch[0]

<tf.Tensor: shape=(128, 42), dtype=float32, numpy=
array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-1.0245436 ,  2.6200242 ,  0.24582736, ..., -0.28313258,
        -1.1932027 ,  0.08073497],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.7584459 ,  2.5239635 ,  0.22189996, ..., -1.0505344 ,
        -1.147827  , -1.4725322 ]], dtype=float32)>

In [27]:
target_batch[0]

<tf.Tensor: shape=(64,), dtype=int32, numpy=
array([60, 22, 23, 19, 19,  0, 36, 32, 50, 51,  0, 39, 32, 45, 35, 50, 39,
       46, 36,  0, 34, 36, 44, 36, 51, 36, 49, 56, 61, 59, 59, 59, 59, 59,
       59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59,
       59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59, 59], dtype=int32)>

In [34]:
REQUIRED_SIGNATURE = "serving_default"
REQUIRED_OUTPUT = "outputs"

# interpreter = tf.lite.Interpreter("model.tflite")
interpreter = tf.lite.Interpreter("../models/model.tflite")

# with open ("/kaggle/input/asl-fingerspelling/character_to_prediction_index.json", "r") as f:
with open ("../data/asl-fingerspelling/character_to_prediction_index.json", "r") as f:
    character_map = json.load(f)

rev_character_map = {j:i for i,j in character_map.items()}
found_signatures = list(interpreter.get_signature_list().keys())

if REQUIRED_SIGNATURE not in found_signatures:
    raise KernelEvalException('Required input signature not found.')

prediction_fn = interpreter.get_signature_runner(REQUIRED_SIGNATURE)

prediction_str = ""
for source_element, target_element in zip(source_batch, target_batch):
    # print(tf.expand_dims(target_element, axis=0).numpy())

    output = prediction_fn(inputs=source_element)

    # print(output[REQUIRED_OUTPUT])

    # break

    print("generated: ", "".join([rev_character_map.get(s, "") for s in np.argmax(output[REQUIRED_OUTPUT], axis=1)]))
    print("target: ", "".join([rev_character_map.get(s, "") for s in target_element.numpy()]))

generated:  +555-55-55-854
target:  7844 east handshoe cemetery
generated:  +55-55-55-55-85
target:  9742 purple aster
generated:  255-555-5559
target:  bani_hamida1
generated:  +55-55-55-55-55
target:  7062 cll culto
generated:  +55-55-55-55-85
target:  +264-3552-4581-49-7671
generated:  +255-55-55-55-55
target:  make-robalo
generated:  +55-55-55-55-55
target:  7441 slippery branch
generated:  +55-55-55-55-85
target:  386 lynds hill
generated:  +55-55-55-55
target:  www.rueckweiler.de
generated:  +65-55-55-55-85
target:  529-003-9827
generated:  +55-55-55-55-85
target:  joshua boyer
generated:  +55-55-55-55-85
target:  +33-368-56-018
generated:  +55-55-55-55-85
target:  julianne clayton
generated:  +59-55-55-55
target:  magdalena mcfarland
generated:  +55-55-55-55-85
target:  clairecmc
generated:  +55-55-55-85-85
target:  2919 camp arbuckle avenue
generated:  +55-55-555-854
target:  davin bryan
generated:  2555 chall road
target:  760-633-2200
generated:  +55-55-55-55
target:  endosco

In [29]:
next(iter(val_dataset))[0][0][0].shape

TensorShape([128, 42])

In [30]:
output = prediction_fn(inputs=next(iter(val_dataset))[0][0][0])

In [31]:
np.argmax(output["outputs"], axis=1)

array([10, 20, 24, 12, 20, 20, 12, 20, 20, 12, 20, 20, 12, 23, 20])

In [32]:
"".join([rev_character_map.get(s, "") for s in np.argmax(output["outputs"], axis=1)])

'+59-55-55-55-85'

In [33]:
interpreter.get_input_details()

[{'name': 'serving_default_inputs:0',
  'index': 0,
  'shape': array([128,  42], dtype=int32),
  'shape_signature': array([-1, 84], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]